In [1]:
df_a = pd.read_csv("web/data/brand_values.csv")
df_a

<IPython.core.display.Javascript object>

,name,value,year,lastValue,rank
0,Apple,214480.0,2018.0,211447.4,1
1,Apple,211447.4,2017.9,208414.8,1
2,Apple,208414.8,2017.8,205382.2,1
3,Apple,205382.2,2017.7,202349.6,1
4,Apple,202349.6,2017.6,199317.0,1
...,...,...,...,...,...
18095,Zara,NaN,2004.5,NaN,100
18096,Zara,NaN,2004.6,NaN,100
18097,Zara,NaN,2004.7,NaN,100
18098,Zara,NaN,2004.8,NaN,100


In [2]:
df = pd.read_csv("web/data/covid-data.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")
df = df[["location", "total_cases", "new_cases"]].reset_index()
df["lastValue"] = df["total_cases"] - df["new_cases"]
df = df.rename(columns={"total_cases" : "value", "location" : "name"})
df = df.fillna(0)
df = df.sort_values(["date", "value"], ascending=[True, False])
df = df.groupby(["name", pd.Grouper(key='date', freq="W-MON")]).sum("value").reset_index()
df["lastValue"] = df["value"].shift().fillna(0)
df = df.drop("new_cases", axis=1)
df = df.rename(columns={"date":"year"})
df["rank"] = 0
df = df[["name", "value", "year", "lastValue", "rank"]]#.to_csv("covid_1.csv")
df["year"] = df["year"].dt.year + df["year"].dt.week/52
df["year"] = df["year"].round(1)
df = df.groupby(["name", "year"]).max().reset_index()
df["lastValue"] = df["value"].shift().fillna(0)
cont = ["Europe", "Asia", "North America", "Africa", "Oceania", "World", "South America", "European Union"]
df = df[~df["name"].isin(cont)]
df["rank"] = df.groupby("year").rank(method="max", ascending=False)["value"].astype(int)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-2-07970d28c589>:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["year"] = df["year"].dt.year + df["year"].dt.week/52


In [3]:
df

,name,year,value,lastValue,rank
0,Afghanistan,2020.2,203.0,0.0,101
1,Afghanistan,2020.3,9537.0,203.0,75
2,Afghanistan,2020.4,96483.0,9537.0,45
3,Afghanistan,2020.5,226442.0,96483.0,44
4,Afghanistan,2020.6,258809.0,226442.0,54
...,...,...,...,...,...
2684,Zimbabwe,2021.0,137606.0,67837.0,111
2685,Zimbabwe,2021.1,245052.0,137606.0,106
2686,Zimbabwe,2021.2,257528.0,245052.0,112
2687,Zimbabwe,2021.3,267681.0,257528.0,113


In [8]:
df.to_csv("web/data/covid_1.csv")

In [5]:
#df.groupby(["name", pd.Grouper(key='date', freq='W-MON')]).sum("value")

dates = df["date"].unique()
df["rank"] = 0
for day in dates:
    for i in range(len(df[df["date"] == day])):
        df[df["date"] == day].iloc[i]["rank"] = i+1

In [6]:
df_a

,name,value,year,lastValue,rank
0,Apple,214480.0,2018.0,211447.4,1
1,Apple,211447.4,2017.9,208414.8,1
2,Apple,208414.8,2017.8,205382.2,1
3,Apple,205382.2,2017.7,202349.6,1
4,Apple,202349.6,2017.6,199317.0,1
...,...,...,...,...,...
18095,Zara,NaN,2004.5,NaN,100
18096,Zara,NaN,2004.6,NaN,100
18097,Zara,NaN,2004.7,NaN,100
18098,Zara,NaN,2004.8,NaN,100


In [7]:
df

,name,year,value,lastValue,rank
0,Afghanistan,2020.2,203.0,0.0,101
1,Afghanistan,2020.3,9537.0,203.0,75
2,Afghanistan,2020.4,96483.0,9537.0,45
3,Afghanistan,2020.5,226442.0,96483.0,44
4,Afghanistan,2020.6,258809.0,226442.0,54
...,...,...,...,...,...
2684,Zimbabwe,2021.0,137606.0,67837.0,111
2685,Zimbabwe,2021.1,245052.0,137606.0,106
2686,Zimbabwe,2021.2,257528.0,245052.0,112
2687,Zimbabwe,2021.3,267681.0,257528.0,113


<ipython-input-13-162520f11f11>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["year"] = df["year"].dt.year + df["year"].dt.week/52


In [17]:
#df["year"] = df["year"].round(1)